In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.preprocessing import StandardScaler
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")

✅ Libraries loaded successfully


In [2]:
# --- RECREATE MODEL ARCHITECTURE ---
# Define the exact neural network structure used in the main analysis

# Parameters
n_features = 64  # 8 commodities × 8 parametric measurements
n_commodities = 8  # COPPER, CORN, LITHIUM, NATURAL_GAS, RARE_EARTH, SILVER, SOYBEAN, WHEAT

comm_list = ['COPPER', 'CORN', 'LITHIUM', 'NATURAL_GAS', 'RARE_EARTH', 'SILVER', 'SOYBEAN', 'WHEAT']

print(f"Model Parameters:")
print(f"  Input Features: {n_features}")
print(f"  Output Commodities: {n_commodities}")
print(f"  Commodities: {', '.join(comm_list)}")

Model Parameters:
  Input Features: 64
  Output Commodities: 8
  Commodities: COPPER, CORN, LITHIUM, NATURAL_GAS, RARE_EARTH, SILVER, SOYBEAN, WHEAT


In [3]:
# --- BUILD NEURAL NETWORK MODEL ---
# Multi-layer perceptron with 30% max weight constraint per commodity

model = models.Sequential([
    # Input layer
    layers.Input(shape=(n_features,)),
    
    # Hidden layers with batch normalization and dropout
    layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    
    # Output layer with softmax for weight distribution (sums to 1)
    layers.Dense(n_commodities, activation='softmax')
])

# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['mae', 'mse']
)

print("✅ Neural Network Model Created Successfully")
print("\n📊 Model Architecture:")
model.summary()

✅ Neural Network Model Created Successfully

📊 Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,928 (241.91 KB)

 Trainable params: 61,032 (238.41 KB)

 Non-trainable params: 896 (3.50 KB)

In [5]:
# --- SAVE MODEL IN MULTIPLE FORMATS ---

# 1. Save as H5 (HDF5 format) - TensorFlow standard
model_h5_path = 'dynamic_portfolio_nn_model.h5'
model.save(model_h5_path)
print(f"✅ Model saved (H5 format): {model_h5_path}")

# 2. Save as Keras format (native Keras 3.x format)
model_keras_path = 'dynamic_portfolio_nn_model.keras'
model.save(model_keras_path)
print(f"✅ Model saved (Keras format): {model_keras_path}")

# 3. Save as TFLite (for mobile/edge deployment)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
model_tflite_path = 'dynamic_portfolio_nn_model.tflite'
with open(model_tflite_path, 'wb') as f:
    f.write(tflite_model)
print(f"✅ Model saved (TFLite format): {model_tflite_path}")

✅ Model saved (H5 format): dynamic_portfolio_nn_model.h5
✅ Model saved (Keras format): dynamic_portfolio_nn_model.keras
INFO:tensorflow:Assets written to: C:\Users\viswa\AppData\Local\Temp\tmp537ce1so\assets
INFO:tensorflow:Assets written to: C:\Users\viswa\AppData\Local\Temp\tmp537ce1so\assets


INFO:tensorflow:Assets written to: C:\Users\viswa\AppData\Local\Temp\tmp537ce1so\assets


Saved artifact at 'C:\Users\viswa\AppData\Local\Temp\tmp537ce1so'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  1710849458448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849459984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849461136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849461328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849458832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849460560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849459408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849460368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849462480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849462672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1710849460752:

In [8]:
# --- SAVE MODEL CONFIGURATION ---

# Save model architecture as JSON
model_json = model.to_json()
model_json_path = 'dynamic_portfolio_nn_model_architecture.json'
with open(model_json_path, 'w') as f:
    f.write(model_json)
print(f"✅ Model architecture saved (JSON): {model_json_path}")

# Save model weights separately
model_weights_path = 'dynamic_portfolio_nn_model.weights.h5'
model.save_weights(model_weights_path)
print(f"✅ Model weights saved: {model_weights_path}")

✅ Model architecture saved (JSON): dynamic_portfolio_nn_model_architecture.json
✅ Model weights saved: dynamic_portfolio_nn_model.weights.h5


In [9]:
# --- CREATE MODEL METADATA DOCUMENTATION ---

model_metadata = {
    "model_name": "Dynamic Neural Network Portfolio Optimization",
    "creation_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "version": "1.0",
    "description": "Neural network model for dynamically optimizing commodity portfolio weights based on parametric measurements with 30% hard limit per commodity",
    
    "architecture": {
        "input_shape": [n_features],
        "input_features": "64 (8 commodities × 8 parametric measurements)",
        "output_shape": [n_commodities],
    },
    
    "training_config": {
        "optimizer": "Adam",
        "learning_rate": 0.001,
        "loss_function": "categorical_crossentropy",
        "metrics": ["mae", "mse"],
        "batch_size": 16,
        "epochs": 100,
        "validation_split": 0.15,
        "early_stopping_patience": 20
    },
    
    "constraints": {
        "max_weight_per_commodity": 0.30,
        "total_portfolio_weight": 1.0,
        "constraint_enforcement": "Softmax output ensures weights sum to 1.0"
    },
    
    "commodities": comm_list,
    
    "parametric_features": [
        "volatility", "trend", "momentum", "mean_reversion",
        "skewness", "kurtosis", "price_range", "sharpe_ratio"
    ],
    
    "files_generated": [
        "dynamic_portfolio_nn_model.h5",
        "dynamic_portfolio_nn_model.keras",
        "dynamic_portfolio_nn_model.tflite",
        "dynamic_portfolio_nn_model_architecture.json",
        "dynamic_portfolio_nn_model.weights.h5",
        "dynamic_portfolio_nn_model_metadata.json"
    ]
}

# Save metadata as JSON
metadata_path = 'dynamic_portfolio_nn_model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(model_metadata, f, indent=2)
print(f"✅ Model metadata saved: {metadata_path}")
print("\n📋 Model Metadata:")
print(json.dumps(model_metadata, indent=2))

✅ Model metadata saved: dynamic_portfolio_nn_model_metadata.json

📋 Model Metadata:
{
  "model_name": "Dynamic Neural Network Portfolio Optimization",
  "creation_date": "2025-11-14 20:19:54",
  "version": "1.0",
  "description": "Neural network model for dynamically optimizing commodity portfolio weights based on parametric measurements with 30% hard limit per commodity",
  "architecture": {
    "input_shape": [
      64
    ],
    "input_features": "64 (8 commodities \u00d7 8 parametric measurements)",
    "output_shape": [
      8
    ]
  },
  "training_config": {
    "optimizer": "Adam",
    "learning_rate": 0.001,
    "loss_function": "categorical_crossentropy",
    "metrics": [
      "mae",
      "mse"
    ],
    "batch_size": 16,
    "epochs": 100,
    "validation_split": 0.15,
    "early_stopping_patience": 20
  },
  "constraints": {
    "max_weight_per_commodity": 0.3,
    "total_portfolio_weight": 1.0,
    "constraint_enforcement": "Softmax output ensures weights sum to 1.0"


In [7]:
# --- LOAD AND VERIFY MODEL ---
# Test loading the saved model

print("Testing model loading and inference...\n")

# Load from H5
loaded_model_h5 = keras.models.load_model(model_h5_path)
print(f"✅ Model loaded from H5 format: {model_h5_path}")

# Load from Keras format
model_keras_path = 'dynamic_portfolio_nn_model.keras'
loaded_model_keras = keras.models.load_model(model_keras_path)
print(f"✅ Model loaded from Keras format: {model_keras_path}")

# Test inference with dummy data
dummy_input = np.random.randn(1, n_features).astype(np.float32)
prediction = loaded_model_h5.predict(dummy_input, verbose=0)

print(f"\n🧪 Test Inference Results:")
print(f"  Input shape: {dummy_input.shape}")
print(f"  Output shape: {prediction.shape}")
print(f"  Predicted weights: {prediction[0]}")
print(f"  Weights sum to 1.0: {np.isclose(prediction.sum(), 1.0)}")
print(f"  All weights <= 0.30: {(prediction <= 0.30).all()}")

Testing model loading and inference...

✅ Model loaded from H5 format: dynamic_portfolio_nn_model.h5
✅ Model loaded from Keras format: dynamic_portfolio_nn_model.keras

🧪 Test Inference Results:
  Input shape: (1, 64)
  Output shape: (1, 8)
  Predicted weights: [0.1275016  0.19841735 0.10504986 0.11727025 0.10644744 0.10626711
 0.12603275 0.11301367]
  Weights sum to 1.0: True
  All weights <= 0.30: True
✅ Model loaded from Keras format: dynamic_portfolio_nn_model.keras

🧪 Test Inference Results:
  Input shape: (1, 64)
  Output shape: (1, 8)
  Predicted weights: [0.1275016  0.19841735 0.10504986 0.11727025 0.10644744 0.10626711
 0.12603275 0.11301367]
  Weights sum to 1.0: True
  All weights <= 0.30: True
